In [2]:
import torch
import lightning as L
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
import torch.nn as nn
import optuna
from optuna.integration import PyTorchLightningPruningCallback

import wandb
from lightning.pytorch.loggers import WandbLogger


In [3]:
class AddDataset(Dataset):
    def __init__(self, data_size=1000000, range_min=-100, range_max=100):
        self.inputs = torch.rand(data_size, 2) * (range_max - range_min) + range_min
        self.targets = (self.inputs[:, 0] <= self.inputs[:, 1]).long()
        self.inputs = self.inputs.float()

    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        y = self.targets[idx]
        return x, y


In [4]:
class DataModule(L.LightningDataModule):
    def __init__(self, batch_size=1024, data_size=100_0000):
        super().__init__()
        self.batch_size = batch_size
        self.data_size = data_size

    def setup(self, stage=None):
        full_dataset = AddDataset(data_size=self.data_size)
        train_size = int(0.8 * len(full_dataset))
        val_size = int(0.1 * len(full_dataset))
        test_size = len(full_dataset) - train_size - val_size

        self.train_ds, self.val_ds, self.test_ds = random_split(
            full_dataset, [train_size, val_size, test_size]
        )

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, shuffle=False)

In [11]:
class Model(L.LightningModule):
    def __init__(self, hidden_size=32,lr=1e-3):
        super().__init__()
        self.save_hyperparameters() 
        self.lr=lr
        self.model=nn.Sequential(nn.Linear(2, hidden_size ),
                                 nn.Tanh(),
                                 nn.Linear(hidden_size, hidden_size),
                                 nn.Tanh(),
                                 nn.Linear(hidden_size,2))
        self.loss_fn = nn.CrossEntropyLoss()
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x) 
        loss = self.loss_fn(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x) 
        loss = self.loss_fn(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        
        return loss
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x) 
        loss = self.loss_fn(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_loss", loss, prog_bar=True)
        
        return loss
    def on_validation_epoch_end(self):
        val_loader = self.trainer.datamodule.val_dataloader()
        x, y = next(iter(val_loader))
        x, y = x.to(self.device), y.to(self.device)
        logits = self(x)
        probs = torch.nn.functional.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        columns = ["Number A", "Number B", "Actual (A<=B)", "Predicted", "Confidence True"]
        data = []
    
        for i in range(10):
            a = x[i][0].item()
            b = x[i][1].item()
            actual = "True" if y[i].item() == 1 else "False"
            predicted = "True" if preds[i].item() == 1 else "False"
            conf = probs[i][1].item() 
            
            data.append([a, b, actual, predicted, conf])
        self.logger.experiment.log({"Validation Samples": wandb.Table(data=data, columns=columns)})
    def predict(self, x):
     
        logits = self(x)
        return F.softmax(logits, dim=1)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [31]:
wandb_logger = WandbLogger(project="absurd-math", name="inequality-classifier-run-")

dataloader=DataModule(data_size=5000, batch_size=1024)
model=Model(hidden_size=64, lr=1e-3)
early_stop = EarlyStopping(monitor="validation_loss", patience=3, mode="min", verbose=True)
trainer = L.Trainer(max_epochs=20, accelerator="auto", devices=1, callbacks=[early_stop], logger=wandb_logger)
trainer.fit(model, datamodule=dataloader)
trainer.test(model, datamodule=dataloader)



💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\loggers\wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 4.5 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
4.5 K     Trainable params
0         Non-trainable params
4.5 K     Total params
0.018     Total estimated model param

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\loops\fit_loop.py:310: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric validation_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.013
`Trainer.fit` stopped: `max_epochs=20` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            1.0            │
│         test_loss         │   0.016651587560772896    │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 1.0, 'test_loss': 0.016651587560772896}]

In [32]:
print(wandb.finish())

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
test_acc,▁
test_loss,▁
train_acc,▁▆██████████████████
train_loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▄▄▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
validation_acc,█▁▆██████████████████
validation_loss,▁█▆▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,20
test_acc,1.0
test_loss,0.01665


None


In [33]:
from optuna.exceptions import TrialPruned

def objective(trial):

    hidden_size = trial.suggest_int("hidden_size", 8, 256)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    n_epochs= trial.suggest_int('epochs', 1,20)
    dm = DataModule(batch_size=batch_size, data_size=5000)
    model = Model(hidden_size=hidden_size, lr=learning_rate)

    wandb_logger = WandbLogger(
        project="absurd-math", 
        group="optuna-tuning", 
        name=f"trial-{trial.number}",
        log_model=False 
    )


    trainer = L.Trainer(
        logger=wandb_logger,
        max_epochs=n_epochs,
        accelerator="auto",
        enable_checkpointing=False, 
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="validation_loss")],
    )

    try:
    
        trainer.fit(model, datamodule=dm)
        return trainer.callback_metrics["validation_loss"].item()

    except Exception as e:
        if not isinstance(e, TrialPruned):
             print(f"Trial failed with error: {e}")
        raise e

    finally:

        wandb.finish()


In [34]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-25 23:39:56,919] A new study created in memory with name: no-name-f20db6e7-98bf-47b3-bbb5-59b608ed19b5
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 1.0 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁
trainer/global_step,▁
validation_acc,▁
validation_loss,▁
epoch,0
trainer/global_step,31
validation_acc,0.964
validation_loss,0.2877


[I 2025-11-25 23:50:25,572] Trial 19 pruned. Trial was pruned at epoch 0.


  hidden_size: 119
  learning_rate: 0.0013880609459959213
  batch_size: 64
  epochs: 3


In [37]:
study.best_params
wandb_logger = WandbLogger(project="absurd-math", name="inequality-classifier-run-")

dataloader=DataModule(data_size=5000, batch_size=study.best_params['batch_size'])
model=Model(hidden_size=study.best_params['hidden_size'], lr=study.best_params['learning_rate'])
early_stop = EarlyStopping(monitor="validation_loss", patience=3, mode="min", verbose=True)
trainer = L.Trainer(max_epochs=study.best_params['epochs'], accelerator="auto", devices=1, callbacks=[early_stop], logger=wandb_logger)
trainer.fit(model, datamodule=dataloader)
trainer.test(model, datamodule=dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 14.9 K | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
14.9 K    Trainable params
0         Non-trainable params
14.9 K    Total params
0.060     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9959999918937683     │
│         test_loss         │    0.01103979628533125    │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9959999918937683, 'test_loss': 0.01103979628533125}]

In [38]:
wandb.finish()

epoch,▁▁▃▃▆▆█
test_acc,▁
test_loss,▁
train_acc,▁██
train_loss,█▁▁
trainer/global_step,▁▁▁▄▄▄███
validation_acc,██▁
validation_loss,▂▁█
epoch,3
test_acc,0.996
test_loss,0.01104


In [39]:
trainer.save_checkpoint('best_model.ckpt')

In [35]:
model.eval()
test_input = torch.tensor([[32193128.,41441.]])
model.predict(test_input)

tensor([[0.9977, 0.0023]], grad_fn=<SoftmaxBackward0>)